In [1]:
import xarray as xr
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import matplotlib as mpl
import cartopy.crs as ccrs

datapath='/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/MF_ML_PREDICT/'
models=['graphcast_','graphcast-oper','pangu_','pangu-oper','fcnv2_dawn','ifs']
references=['era','ifs_init']
figpath='/users/mfeldman/figs/spatial/'
c1='#648fff' #lightblue
c2='#785ef0' #indigo
c3='#dc267f' #magenta
c4='#fe6100' #orange
c5='#ffb000' #gold
c6='#000000' #black



In [ ]:
era_ref=xr.open_dataset(datapath+'USA_tor_2020_era.nc').sortby('latitude').fillna(0)
cape=era_ref.cape.squeeze().values
wms=(cape * 2)**0.5 * era_ref.bs_06
era_ref=era_ref.assign(wms=lambda era_ref: wms)
nn=0
mm=0
for mm in range(6)[:-1]:
    var=['cape','cin','bs_06','bs_03','bs_01','wms'][mm]
    tit=['CAPE [J kg$^{-1}$]','CIN [J kg$^{-1}$]','0-6km shear [m s$^{-1}$]','0-3km shear [m s$^{-1}$]',
         '0-1km shear [m s$^{-1}$]','wmax-shear [m$^2$ s$^{-2}$]'][mm]
    vm=[3000,500,30,20,10,1000][mm]
    vl=[300,10,5,1,1,100][mm]
    l1=[300,-100,10,5,5,300][mm]
    l2=[1000,-300,20,10,10,500][mm]
    f1=[1,-1,1,1,1,1][mm]
    # fig,axes = plt.subplots(1,3)
    # fig2,axes2 = plt.subplots(1,3,figsize=(12, 5))
    for nn in range(len(models)):
        kw=['lat','lat','latitude','latitude','latitude','latitude'][nn]
        kw1=['lon','lon','longitude','longitude','longitude','longitude'][nn]
        kw2=['prediction_timedelta','prediction_timedelta','prediction_timedelta','prediction_timedelta','time','prediction_timedelta'][nn]
        
        model=models[nn]
        color=[c1,c2,c3,c4,c5,c6][nn]
        print(model)

        files=sorted(glob(datapath+'USA_tor_2020_'+model+'*2020*.nc'),reverse=True)
       

        for file in range(len(files)-2):
            print(file)
            model_set=xr.open_dataset(files[file]).sortby(kw).fillna(0).squeeze()
            init=files[file][-13:-3]
            #d_model_set = model_set - era_ref
            # cm=plt.cm.Reds
            # cm.set_under(color="grey")
            # cm.set_bad(color="grey")
            cmd=plt.cm.seismic
            print('making plot')
            p = model_set[var].sel(**{kw: slice(20,40)},**{kw1: slice(260,290)}).plot(x=kw1, y=kw, col=kw2, col_wrap=4,
                        transform=ccrs.PlateCarree(), subplot_kws={'projection':ccrs.PlateCarree()},
                        cmap=cm,vmin=vl,vmax=vm,levels=np.arange(vl,vm,np.ceil(vm/20)),extend='both')# col_wrap=4)
            for ax in p.axs.flat:
                            ax.coastlines()
            f=p.fig
            f.savefig(figpath+'plot_'+model+'_'+init+'_'+var+'.png',  dpi=300, bbox_inches='tight')
            plt.close()
            print('computing delta')
            d_model_set = model_set[var].sel(**{kw: slice(20,40)},**{kw1: slice(260,290)}).squeeze() - \
                            era_ref[var].sel(latitude=slice(20,40),longitude=slice(260,290)).squeeze().values
                
            print('making plot')
            
            p = d_model_set.plot(x=kw1, y=kw, col=kw2, col_wrap=4,
                        transform=ccrs.PlateCarree(), subplot_kws={'projection':ccrs.PlateCarree()},
                        cmap=cmd,vmin=-vm,vmax=vm,levels=np.arange(-vm,vm,np.ceil(2*vm/20)),extend='both')# col_wrap=4)
            for ax in p.axs.flat:
                            ax.coastlines()
            f=p.fig
            f.savefig(figpath+'deltaplot_'+model+'_'+init+'_'+var+'.png',  dpi=300, bbox_inches='tight')
            plt.close()
            
            # p = d_model_set[var].sel(latitude=slice(20,40),longitude=slice(260,290)).plot(x="longitude", y="latitude", col="time", col_wrap=4,transform=ccrs.PlateCarree(), subplot_kws={'projection':ccrs.PlateCarree()}\
            #             ,cmap=cmd,vmin=-vm,vmax=vm,levels=np.arange(-vm,vm,int(2*vm/20)),extend='both')# col_wrap=4)
            # for ax in p.axs.flat:
            #                 ax.coastlines()
            # f=p.fig
            # f.savefig(figpath+'/spatial/deltaplot_'+model+'_'+init+'_'+var+'.png',  dpi=300, bbox_inches='tight')
            # plt.close()


    #     axes2[0].plot(np.arange(i+2*len(files))*6,np.nanmedian(sal_s,axis=1),c=color,label=model)
    #     axes2[0].fill_between(np.arange(i+2*len(files))*6,np.nanmax(sal_s,axis=1),np.nanmin(sal_s,axis=1),facecolor=color,alpha=0.3)
    #     axes2[0].set_title('structure')
    #     axes2[0].set_xlabel('leadtime [h]')
    #     axes2[0].set_ylabel('score [-2,2]')
    #     axes2[0].legend()
    #     axes2[1].plot(np.arange(i+2*len(files))*6,np.nanmedian(sal_a,axis=1),c=color,label=model)
    #     axes2[1].fill_between(np.arange(i+2*len(files))*6,np.nanmax(sal_a,axis=1),np.nanmin(sal_a,axis=1),facecolor=color,alpha=0.3)
    #     axes2[1].set_title('amplitude')
    #     axes2[1].set_xlabel('leadtime [h]')
    #     axes2[1].set_ylabel('score [-2,2]')
    #     axes2[2].plot(np.arange(i+2*len(files))*6,np.nanmedian(sal_l,axis=1),c=color)
    #     axes2[2].fill_between(np.arange(i+2*len(files))*6,np.nanmax(sal_l,axis=1),np.nanmin(sal_l,axis=1),facecolor=color,alpha=0.3)
    #     axes2[2].set_title('location')
    #     axes2[2].set_xlabel('leadtime [h]')
    #     axes2[2].set_ylabel('score [0,2]')
    #     fig2.suptitle('SAL score '+str(l1)+' '+tit)


    # fig2.tight_layout()
    # fig2.show()
    # fig2.savefig(figpath+var+'_sal_scores.png')


graphcast_
0
making plot
computing delta
making plot
1
making plot
computing delta
making plot
2
making plot
computing delta
making plot
3
making plot
computing delta
making plot
4
making plot
computing delta
making plot
5
making plot
computing delta
making plot
6
making plot
computing delta
making plot
7
making plot
computing delta
making plot
8
making plot
computing delta
making plot
9
making plot
computing delta
making plot
10
making plot
computing delta
making plot
11
making plot
computing delta
making plot
12
making plot
computing delta
making plot
13
making plot
computing delta
making plot
14
making plot
computing delta
making plot
15
making plot
computing delta
making plot
graphcast-oper
0
making plot
computing delta
making plot
1
making plot
computing delta
making plot
2
making plot
computing delta
making plot
3
making plot
computing delta
making plot
4
making plot
computing delta
making plot
5
making plot
computing delta
making plot
6
making plot
computing delta
making plot
7


In [25]:
np.ceil(vm/20)

1.0

In [8]:
files[:-2]

['/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/MF_ML_PREDICT/USA_tor_2020_graphcast_2020041112.nc',
 '/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/MF_ML_PREDICT/USA_tor_2020_graphcast_2020041100.nc',
 '/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/MF_ML_PREDICT/USA_tor_2020_graphcast_2020041012.nc',
 '/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/MF_ML_PREDICT/USA_tor_2020_graphcast_2020041000.nc',
 '/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/MF_ML_PREDICT/USA_tor_2020_graphcast_2020040912.nc',
 '/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/MF_ML_PREDICT/USA_tor_2020_graphcast_2020040900.nc',
 '/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/MF_ML_PREDICT/USA_tor_2020_graphcast_2020040812.nc',
 '/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/MF_ML_PREDICT/USA_tor_2020_graphcast_2020040800.nc',
 '/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/MF_ML_PREDICT/USA_tor_2020_graphcast_2020040712.nc',
 '/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/MF_ML_PREDICT/USA_tor_2020_graphc